In Module 0, we mainly focused on setting up the foundations for analyzing languages as formal objects. We defined a language $L$ on an alphabet $\Sigma$ as a subset of the set of strings $\Sigma^* = \bigcup_{i=0}^\infty \Sigma^i$ on $\Sigma$ (i.e. $L \in 2^{\Sigma^*}$). We explored one way of describing languages in the form of regular expressions on $\Sigma$ and we discussed one way of describing a relation between strings in the form of minimum edit distance. 

(In fact, we decribed minimum edit distance as a relation from tuples of strings to distances; but as we explored in Assignment 3, we can also think of a distance, in the context of a particular weighting on edit operations, as a relation on strings: namely, a relation on strings that are \{at most, exactly, at least, ...\} that distance apart. This idea extends the notion of a [ball](https://en.wikipedia.org/wiki/Ball_(mathematics)) or [sphere](https://en.wikipedia.org/wiki/Sphere), depending on the constraint, to strings.)

# Levels of Abstraction

What we will now begin to do is to deploy these tools for describing and expressing generalizations about possible languages. To do this, we are going to need to start thinking in terms of sets of sets of languages. That is, we will begin to think about subsets $\mathcal{L}$ of the set of languages $2^{\Sigma^*}$ on $\Sigma$. This approach will yield four important levels of abstraction:

1. Primitive (unanalyzed) elements $\sigma \in \Sigma$
2. Collections of primitive elements: sets $\Sigma$ or sequences $\boldsymbol\sigma \in \Sigma^*$
3. Languages: collections of sequences of primitive elements $L \in 2^{\Sigma^*}$
4. Collections of languages $\mathcal{L} \subseteq 2^{\Sigma^*}$

We are going to be particularly interested in collections of languages that share some interesting properties. We will call such collections *classes of languages* (or *families of languages*), and we will be interested in ways of compactly describing those classes that leverage the property shared by languages in the class. We will call compact descriptions of a particular language grammars and collections thereof classes of grammars $\mathcal{G}$. 

# Generation

We will characterize the relationship between grammars and languages as one of *generation*: a grammars $G$ generates a language $L$ if $G$ is a description of that language. To express that $G$ generates $L$, we will say that $\mathbb{L}(G) = L$, where $\mathbb{L}$ is some function from grammars to languages. (It is important to note that generation sounds like a [procedural](https://en.wikipedia.org/wiki/Procedural_programming) concept, but it is really [declarative](https://en.wikipedia.org/wiki/Declarative_programming). Know that $G$ generates a language $L$ does not require us to know how to build $L$ from $G$.)

## Example: regular expressions

We've already seen one kind of grammar under this definition: regulars expressions. Remember that regular expressions $R(\Sigma)$ on $\Sigma$ themselves are strings of a language on $\Sigma \cup\{\epsilon, \emptyset, \cup, \circ, (, ), *\}$. We formally define these strings recursively.

$\rho$ is a regular expression if and only if:

1. $\rho \in \Sigma \cup \{\epsilon, \emptyset\}$
2. $\rho$ is $(\rho_1 \cup \rho_2)$ for some regular expressions $\rho_1$ and $\rho_2$
3. $\rho$ is $(\rho_1 \circ \rho_2)$ for some regular expressions $\rho_1$ and $\rho_2$
4. $\rho$ is $\rho_1^*$ for some regular expression $\rho_1$

We can generate all the regular expressions given an alphabet.

In [ ]:
def regular_expressions(sigma):
    old_regex_set = frozenset(sigma | {'∅', '𝜖'})
    
    for rho in old_regex_set:
        yield rho
    
    while True:
        new_regex_set = set(old_regex_set)
            
        for rho in old_regex_set:
            elem = (rho, '*')
            new_regex_set |= {elem}
            yield elem
            
        for rho1 in old_regex_set:
            for rho2 in old_regex_set:
                elem = (rho1, '∪', rho2)
                new_regex_set |= {elem}
                yield elem
                
        for rho1 in old_regex_set:
            for rho2 in old_regex_set:
                elem = (rho1, '∘', rho2)
                new_regex_set |= {elem}
                yield elem
                
        old_regex_set = frozenset(new_regex_set)

for i, r in enumerate(regular_expressions({'ə', 'm'})):
    print(r)
    
    if i > 10:
        break

∅
m
𝜖
ə
('∅', '*')
('m', '*')
('𝜖', '*')
('ə', '*')
('∅', '∪', '∅')
('∅', '∪', 'm')
('∅', '∪', '𝜖')
('∅', '∪', 'ə')


Regular expressions (so defined) *evaluate* to sets of strings on $\Sigma$–i.e. languages on $\Sigma$. Another way of thinking about this is that a regular expression on $\Sigma$ *describes* a language on $\Sigma$.

We can define this evaluation procedure formally as a function $\text{eval}: R(\Sigma) \rightarrow 2^{\Sigma^*}$, where $R(\Sigma)$ is the set of regular expressions on $\Sigma$.

$\text{eval}(\rho) = \begin{cases}\{\} & \text{if } \rho = \emptyset \\\{\_\} & \text{if } \rho = \epsilon \\ \{\rho\} & \text{if } \rho \in \Sigma\\ \text{eval}(\rho_1) \times \text{eval}(\rho_2) & \text{if } \rho = (\rho_1 \circ \rho_2) \\ \text{eval}(\rho_1) \cup \text{eval}(\rho_2) & \text{if } \rho = (\rho_1 \cup \rho_2)\\  \bigcup_{i = 0}^\infty \text{eval}(\rho_1)^i & \text{if } \rho = \rho_1^*\\ \end{cases}$

In [ ]:
def evaluate_regular_expression(regex):
    if regex == '∅':
        return
    
    elif regex == '𝜖':
        yield ''
    
    elif isinstance(regex, str):
        yield regex
    
    elif regex[1] == '*':
        i = 0
        while True:
            for s in evaluate_regular_expression(regex[0]):
                yield s*i
            
            i += 1
            
    elif regex[1] == '∪':
        for s1 in evaluate_regular_expression(regex[0]):
            yield s1
            
        for s2 in evaluate_regular_expression(regex[2]):
            yield s2
            
    elif regex[1] == '∘':
        for s1 in evaluate_regular_expression(regex[0]):
            for s2 in evaluate_regular_expression(regex[2]):
                yield s1 + s2

for i, s in enumerate(evaluate_regular_expression(('ə', '∘', ('m', '∪', 'g')))):
    if i < 10:
      print(s)

əm
əg


Each regular expression is thus a grammar; the set of all regular expressions is a class of grammars; and $\text{eval}$ is an implementation of $\mathbb{L}$. We will call the class of languages $\mathcal{R}$ generated by the regular expressions the *regular languages*: $$\mathcal{R} \equiv \mathbb{L}(R(\Sigma)) = \{\mathbb{L}(r) \mid r \in R(\Sigma)\} = \{\text{eval}(r) \mid r \in R(\Sigma)\}$$

We will be studying these languages in depth, because as I've mentiond before and as you will read about in Heinz 2018, it seems very likely that all phonological grammars of natural languages are a subset of this class. To study these languages, it will be useful to introduce a class of grammars that are equivalent to the regular expressions in that they generate exactly the same set of languages–i.e. the regular languages. These grammars are known as *finite state automata*.

We will refer to this sort of equivalence–i.e. that $\mathbb{L}(\mathcal{G}_1) = \mathbb{L}(\mathcal{G}_2)$ for classes $\mathcal{G}_1$ and $\mathcal{G}_2$–as *weak equivalence* or *equivalence in weak generative capacity*. We will discuss another notion–*strong equivalence*–later. First, though we should discuss the broader context.

## The Generativist Conceit

Why would we want to do all this? In the first lecture, I discussed what I called the *Generativist Conceit*: that we can understand natural language by studying the class of grammars that generate all + only the classes of natural languages. Though certain strong ideas associated with the Generativist Conceit are controversial, as I have stated it (relatively weakly) here, it is relatively uncontroversial: if our aim is to provide a scientific theory that characterizes natural language (whatever that might be: a colection of actual utterances, an aspect of human psychology or some abstract object with an existence independent of human minds), we at least need some method for encoding that theory in a way that makes testable predictions. Grammars are definitionally such a method as I've described them.

Another relatively uncontroversial observation that drives the Generativist Conceit is that children must be able to start without a language and come to know one, and the one they come to know is determined in some important way from their environment (rather than their genetics) in a finite amount of time on the basis of a finite number of observations of actual language use. Why this observation is important is that it means that natural languages must be *learnable* from a finite number of examples. Grammars are usually finite objects that (may) describe countably infinite languages, and so they might be a useful tool for describing the process by which a child comes to know a language.

### Learning

More formally, we may say that children have some method $c$ for mapping sequences of strings (or *texts*) $\mathbf{t} \in \Sigma^{**}$ along with some extralinguistic experience $E \in \mathcal{E}$ into a grammar $G \in \mathcal{G}$: $$c: \Sigma^{**} \times \mathcal{E} \rightarrow \mathcal{G}$$. A child's extralinguistic experience is nontrivial to represent, so to make the problem more tractable, we will often ignore that part.

To demonstrate that the problem of learning a grammar from a text, consider the following formal model, known as [*language identification in the limit*](https://en.wikipedia.org/wiki/Language_identification_in_the_limit), due to [Gold 1964](https://www.rand.org/pubs/research_memoranda/RM4136.html). 

We start from the idea that a *learner* should be modeled as a function $f: \Sigma^{**} \rightarrow 2^{\Sigma^*}$ that maps an *environment* $E = \langle \boldsymbol\sigma_1, \boldsymbol\sigma_2, \ldots \rangle \in \Sigma^{**}$ to a language $L$, where an environment is a sequence of strings in $L$. We will say that a learner $f$ learns a language $L$ given an *environment* (or *text*) $E \subset L^*$ if the learner outputs $L$ after seeing enough examples from the environment and that it learns a language $L$ if it learns that language in any environment. That is, $f$ learns $L$ if it doesn't matter which sequence you give it: $f \text{ learns } L \leftrightarrow \forall \mathbf{e} \in L^*: f(\mathbf{e}) = L$. A language family $\mathcal{L}$ is learnable if there exists a language learner that can learn all languages in the family: $f \text{ learns } \mathcal{L} \leftrightarrow \forall L \in \mathcal{L}: \forall \mathbf{e} \in L^*: f(\mathbf{e}) = L$.

[Gold (1967)](https://www.sciencedirect.com/science/article/pii/S0019995867911655) showed that, if a language family $\mathcal{L}$ contains languages $L_1, L_2, \ldots, L_\infty$, such that $L_1 \subset L_2 \subset \ldots \subset L_\infty \bigcup _{i=1}^\infty L_i$, then it is not learnable. Here's the idea behind the proof: suppose $f$ is a learner that can learn $L_1, L_2, \ldots, L_\infty$; we'll show that it cannot learn $L_\infty$, by constructing an environment for $L_\infty$ that "tricks" $f$. 

First, we construct environments $E_1, E_2, \ldots$ such that $f(E_i) = L_i$. Next, we construct environment $E_\infty$ for $L_\infty$ inductively as follows:

1. Present $f$ with $E_1$ until it outputs $L_{1}$.
2. Switch to presenting $f$ with an environment that alternates the rest of $E_1$ and the entirety of $E_2$. Since $L_1 \subset L_2$, this concatenated environment is still an environment for $L_2$, so $f$ must eventually output $L_2$.
3. Switch to presenting the rest of $E_1 \circ E_2$ and the entirety of $E_3$ alternatively. And so on for all $i$.

By construction, the resulting environment $E$ contains the entirety of $E_{1},E_{2},\ldots$, thus it contains $L_\infty$, so it is an environment for $L_\infty$. But since the learner always switches to $L_{i}$ for some finite $i$, it never converges to $L_{\infty}$.

The point here is that we need to be careful about the class of languages we pick out as possible ones the learner might select from. We'll explore this idea in the context of finite state automata, which are equivalent to regular expressions. Importantly, the full class of finite state automata have the problematic containment structure above, but the hope is that if we constrain these grammars in particular ways, we can avoid these sorts of issues while retaining coverage of natural languages.

# Finite State Automata

We'll start with an application of [FSAs to modeling licit English syllables](https://drive.google.com/file/d/1OreZ2dUdyDYRIKet7bzn617rYDG2NbIz/view?usp=share_link) to get a sense for what they look like, then we'll formalize them. To visualize FSAs, we'll use [`pynini`](https://www.openfst.org/twiki/bin/view/GRM/Pynini) (We'll also use `pynini` later when we define finite state transducers.)

In [ ]:
! pip install --only-binary :all: pynini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 MB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install wurlitzer

%load_ext wurlitzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pynini



## Deterministic Finite State Automaton

A Deterministic Finite State Automaton (DFA) is a grammar with 5 components:

1. A set of states $Q$
2. An alphabet $\Sigma$
3. A transition function $\delta : Q \times \Sigma \rightarrow Q$
4. An initial state $q_0 \in Q$
5. A set of final states $F \subseteq Q$

## Nondeterministic Finite State Automaton

A Nondeterministic Finite State Automaton (NFA) is also a grammar with 5 components:

1. A set of states $Q$
2. An alphabet $\Sigma$
3. A transition function $\delta : Q \times (\Sigma\,\color{red}{\cup \{\epsilon\}}) \rightarrow \color{red}{\mathcal{P}(Q)}$
4. An initial state $q_0 \in Q$
5. A set of final states $F \subseteq Q$


In [ ]:
from copy import copy, deepcopy
from typing import Union, Optional, Dict, List
from functools import lru_cache

class TransitionFunction:
    """A finite state machine transition function

    Parameters
    ----------
    transition_graph

    Attributes
    ----------
    isdeterministic
    istotalfunction
    transition_graph

    Methods
    -------
    validate(alphabet, states)
    relable_states(state_map)
    totalize()
    """

    def __init__(self, transition_graph: Dict[Tuple[str, str], Set[str]]):
        self._transition_graph = transition_graph

    def __call__(self, state: str, symbol: str) -> Set[str]:
        try:
            return self._transition_graph[(state, symbol)]
        except KeyError:
            return set({})

    def __or__(self, other):
        return TransitionFunction(dict(self._transition_graph, 
                                       **other._transition_graph))
        
    def add_transitions(self, transition_graph):
        self._transition_graph.update(transition_graph)

    def validate(self, alphabet, states):
        self._validate_input_values(alphabet, states)
        self._validate_output_types()
        self._homogenize_output_types()
        self._validate_output_values(states)

    def _validate_input_values(self, alphabet, states):
        for state, symbol in self._transition_graph.keys():
            try:
                assert symbol in alphabet

            except AssertionError:
                msg = 'all input symbols in transition function ' +\
                      'must be in alphabet'
                raise ValueError(msg)

            try:
                assert state in states

            except AssertionError:
                msg = 'all input states in transition function ' +\
                      'must be in set of states'
                raise ValueError(msg)

    def _validate_output_types(self):
        for states in self._transition_graph.values():
            try:
                t = type(states)
                assert t is str or t is set

            except AssertionError:
                msg = 'all outputs in transition function' +\
                      'must be specified via str or set'
                raise ValueError(msg)            

    def _homogenize_output_types(self):
        outputs = self._transition_graph.values()

        for inp, out in self._transition_graph.items():
            if type(out) is str:
                self._transition_graph[inp] = {out}

    def _validate_output_values(self, states):
        for out in self._transition_graph.values():
            try:
                assert all([state in states for state in out])
            except AssertionError:
                msg = 'all output symbols in transition function' +\
                      'must be in states'
                raise ValueError(msg)

    @property
    def transition_graph(self):
        return self._transition_graph

class FiniteStateAutomaton:
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """

    def __init__(self, alphabet: Set[str], states: Set[str], 
                 initial_state: str, final_states: Set[str], 
                 transition_graph: 'TransitionFunction'):
        self._alphabet = {''} | alphabet
        self._states = states
        self._initial_state = initial_state
        self._final_states = final_states
        self._transition_function = TransitionFunction(transition_graph)

        self._validate_initial_state()
        self._validate_final_states()
        self._transition_function.validate(self._alphabet, states)

        self._generator = self._build_generator()

    def __iter__(self):
        return self

    def __next__(self):
        return next(self._generator)

    def _build_generator(self):
        string_buffer = [(self._initial_state, '')]
        
        if self._initial_state in self._final_states:
            stack = ['']
        else:
            stack = []

        while string_buffer:
            if stack:
                yield stack.pop()

            else:
                # this is very inefficient when we have total
                # transition functions with many transitions to the
                # sink state; could be optimized by removing a string
                # if it's looping in a sink state, but we don't know
                # in general what the sink state is
                new_buffer = []
                for symb in self._alphabet:
                    for old_state, string in string_buffer:
                        new_states = self._transition_function(old_state, symb)
                        for st in new_states:
                            new_elem = (st, string+symb)
                            new_buffer.append(new_elem)

                stack += [string
                          for state, string in new_buffer
                          if state in self._final_states]

                string_buffer = new_buffer


We often define (and draw) FSAs s.t. their transition functions are partial. We can assume all FSA transition functions $\delta$ are in fact total by adding a *sink state* $q_\text{sink} \not\in F$ to the FSA and mapping all $\langle q, \sigma \rangle$ pairs for which $\delta$ is undefined to $q_\text{sink}$.

In [ ]:
class TransitionFunction(TransitionFunction):
    """A finite state machine transition function

    Parameters
    ----------
    transition_graph

    Attributes
    ----------
    isdeterministic
    istotalfunction
    transition_graph

    Methods
    -------
    validate(alphabet, states)
    relable_states(state_map)
    totalize()
    """

    def istotalfunction(self, states, alphabet):
        return all((s, a) in self._transition_graph
                    for s in states
                    for a in alphabet)

    def totalize(self, states, alphabet):
        if not self.istotalfunction(states, alphabet):
            domain = {(s, a) for s in states for a in alphabet}

            sink_state = 'qsink'

            while sink_state in states:
                sink_state += 'sink'

            for inp in domain:
                if inp not in self._transition_graph:
                    self._transition_graph[inp] = {sink_state}

It turns out that these two ways of defining FSAs are at least weakly equivalent. That is, the class of languages generated by NFAs is the same as the class generated by DFAs; therefore, both generate exactly the regular languages.

## DFAs are (strongly) equivalent to NFAs 

Every DFA can be converted to a strongly equivalent NFA. A DFA $G = \langle Q, \Sigma, \delta, q_0, F \rangle$ can be converted to an NFA $G' = \langle Q', \Sigma, \delta', q'_0, F' \rangle$ by defining the $G'$ transition function $\delta'(q,\sigma) = \{\delta(q,\sigma)\}$

In [ ]:
class TransitionFunction(TransitionFunction):
    """A finite state machine transition function

    Parameters
    ----------
    transition_graph

    Attributes
    ----------
    isdeterministic
    istotalfunction
    transition_graph

    Methods
    -------
    validate(alphabet, states)
    relable_states(state_map)
    totalize()
    """

    @property
    def isdeterministic(self):
        return all(len(v) < 2 for v in self._transition_graph.values())

class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """

    @property
    def isdeterministic(self) -> bool:
        return self._transition_function.isdeterministic



## NFAs are (at least weakly) equivalent to DFAs

Every NFA can be converted to a weakly equivalent DFA

The $\epsilon$-closure of $\delta$ is $$\delta_\epsilon(q, \sigma) =
\begin{cases}
\emptyset & \text{if }  \delta(q, \sigma) \text{ is undefined}\\
\delta(q, \sigma) \cup \bigcup_{r \in \delta(q, \sigma)} \delta_\epsilon(r, \epsilon) & \text{otherwise} \\
\end{cases}$$

An NFA $G = \langle Q, \Sigma, \delta, q_0, F \rangle$ can be converted to a DFA $G' = \langle Q', \Sigma, \delta', q'_0, F' \rangle$ by defining:

1. $Q' = \mathcal{P}(Q)$
2. $\delta'(R, \sigma) = \bigcup_{r\in R}\delta_\epsilon(r, \sigma)$
3. $q'_0 = \{q_0\} \cup \delta_\epsilon(q_0, \epsilon)$
4. $F' = \{R\;|\; \exists r \in R: r \in F\}$



In [ ]:
from functools import reduce
from itertools import chain, combinations

from typing import Set, Tuple

def powerset(iterable):
    """https://docs.python.org/3/library/itertools.html#recipes"""
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def transitive_closure(edges: Set[Tuple[str]]) -> Set[Tuple[str]]:
    """
    the transitive closure of a graph

    Parameters
    ----------
    edges
        the graph to compute the closure of

    Returns
    ----------
    set(tuple)
    """
    while True:
        new_edges = {(x, w)
                     for x, y in edges
                     for q, w in edges
                     if q == y}

        all_edges = edges | new_edges

        if all_edges == edges:
            return edges

        edges = all_edges

class TransitionFunction(TransitionFunction):
    """A finite state machine transition function

    Parameters
    ----------
    transition_graph

    Attributes
    ----------
    isdeterministic
    istotalfunction
    transition_graph

    Methods
    -------
    validate(alphabet, states)
    relable_states(state_map)
    totalize()
    """

    def validate(self, alphabet, states):
        self._validate_input_values(alphabet, states)
        self._validate_output_types()
        self._homogenize_output_types()
        self._validate_output_values(states)

        self._add_epsilon_transitive_closure()

    def _add_epsilon_transitive_closure(self):
        # get the state graph of all epsilon transitions
        transitions = {(instate, outstate)
                       for (instate, insymb), outs in self._transition_graph.items()
                       for outstate in outs if not insymb}
        
        # compute the transitive closure of the epsilon transition
        # state graph; requires homogenization beforehand
        for instate, outstate in transitive_closure(transitions):
            self._transition_graph[(instate, '')] |= {outstate}

        new_graph = dict(self._transition_graph)

        # add alphabet transitions from all states q_i that exit
        # with an alphabet transition (q_i, a) and enter states q_j
        # with epsilon transitions to states q_k
        for (instate1, insymb1), outs1 in self._transition_graph.items():
            for (instate2, insymb2), outs2 in self._transition_graph.items():
                if instate2 in outs1 and not insymb2:
                    # vacuously adds the already added epsilon
                    # transitions as well
                    try:
                        new_graph[(instate1,insymb1)] |= outs2
                    except KeyError:
                        new_graph[(instate1,insymb1)] = outs2

        self._transition_graph = new_graph

    def determinize(self, initial_state):
        new_initial_state = {initial_state} | self(initial_state, "")
        new_transition_graph = {(instate, insymb): outstates
                                for (instate, insymb), outstates 
                                in self._transition_graph.items()
                                if insymb}
        
        return new_initial_state, TransitionFunction(new_transition_graph)


class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """
        
    def determinize(self) -> 'FiniteStateAutomaton':
        """
        if nondeterministic, determinize the FSA

        Returns
        -------
        FiniteStateAutomaton
        """
        new_states = {'_'.join(sorted(s)) for s in powerset(self._states) if s}
        # doesn't handle epsilons coming from the start state
        new_init, new_trans = self._transition_function.determinize(self._initial_state)
        new_final = {'_'.join(sorted(s)) for s in powerset(self._states)
                     if any([t in s for t in self._final_states])
                     if s}
        
        new_transition = {('_'.join(sorted(s)), a): {'_'.join(sorted(t))}
                          for s in powerset(self._states)
                          for t in powerset(self._states)
                          for a in self._alphabet
                          if any([tprime in new_trans(sprime,a)
                                  for sprime in s for tprime in t])
                          if s and t}

        return FiniteStateAutomaton(self._alphabet-{''}, new_states,
                                    '_'.join(sorted(new_init)), new_final,
                                    new_transition)

    @property
    def isdeterministic(self) -> bool:
        return self._transition_function.isdeterministic



# The regular operations

## Union

Given NFAs $M_1 = \langle Q_1, \Sigma_1, \delta_1, q_1, F_1 \rangle$ recognizing $A = \mathbb{L}(M_1)$ and $M_2 = \langle Q_2, \Sigma_2, \delta_2, q_2, F_2 \rangle$ recognizing $B = \mathbb{L}(M_2)$, we construct $M = \text{union}(M_1, M_2) = \langle Q, \Sigma, \delta, q_0, F \rangle$ recognizing $A \cup B = \mathbb{L}(M) = \mathbb{L}(\text{union}(M_1, M_2))$:

1. Relabel $Q_1$ and $Q_2$ so they are mutually exclusive and do not contain $q_0$; $Q = Q'_1 \cup Q'_2 \cup \{q_0\}$ and $\Sigma = \Sigma_1 \cup \Sigma_2$
2. Define $$\delta(q, \sigma) = \begin{cases}
\{q_1, q_2\} & \text{if } q=q_0 \land \sigma=\epsilon \\
\delta_1(q, \sigma) & \text{if } q\in Q_1 \\
\delta_2(q, \sigma) & \text{if } q\in Q_2 \\
\text{undefined} & \text{otherwise} \\
\end{cases}$$
3. Define $F = F_1 \cup F_2$

In [ ]:
class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """

    def __or__(self, other):
        return self.union(other)

    def _deepcopy(self):
        fsa = copy(self)
        del fsa._generator

        return deepcopy(fsa)
        
    def _relabel_fsas(self, other):
        """
        append tag to the input/ouput states throughout two FSAs

        Parameters
        ----------
        other : FiniteStateAutomaton
        """

        fsa1 = self._deepcopy()._relabel_states(str(id(self)))
        fsa2 = other._deepcopy()._relabel_states(str(id(other)))

        return fsa1, fsa2

    def _relabel_states(self, tag):
        """
        append tag to the input/ouput states throughout the FSA

        Parameters
        ----------
        tag : str
        """

        state_map = {s: s+'_'+tag for s in self._states}
        
        self._states = {state_map[s] for s in self._states}    
        self._initial_state += '_'+tag
        self._final_states = {state_map[s] for s in self._final_states}

        self._transition_function = self._transition_function
        self._transition_function.relabel_states(state_map)
        
        return self

    def union(self, other: 'FiniteStateAutomaton') -> 'FiniteStateAutomaton':
        """
        union this FSA with another

        Parameters
        ----------
        other : FiniteStateAutomaton

        Returns
        -------
        FiniteStateAutomaton
        """
        msg = "you still need to implement FiniteStateAutomaton.union"
        raise NotImplementedError(msg)



## Concatenation

Given NFAs $M_1 = \langle Q_1, \Sigma_1, \delta_1, q_1, F_1 \rangle$ recognizing $A = \mathbb{L}(M_1)$ and $M_2 = \langle Q_2, \Sigma_2, \delta_2, q_2, F_2 \rangle$ recognizing $B = \mathbb{L}(M_2)$, we construct $M = \text{concatenate}(M_1, M_2) = \langle Q, \Sigma, \delta, q_1, F_2 \rangle$ recognizing $A \circ B = \mathbb{L}(M) = \mathbb{L}(\text{concatenate}(M_1, M_2))$:

1. Relabel $Q_1$ and $Q_2$ so they are mutually exclusive; $Q = Q'_1 \cup Q'_2$ and $\Sigma = \Sigma_1 \cup \Sigma_2$
2. Define $$\delta(q, \sigma) = \begin{cases}
\delta_1(q, \sigma) & \text{if } q\in Q_1 \land q\not\in F_1 \\
\delta_1(q, \sigma) & \text{if } q\in F_1 \land \sigma \neq \epsilon \\
\delta_1(q, \sigma) \cup \{q_2\} & \text{if } q\in F_1 \land \sigma = \epsilon \\
\delta_2(q, \sigma)& \text{if } q\in Q_2 \\
\end{cases}$$


In [ ]:
class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """
        
    def __add__(self, other):
        return self.concatenate(other)

    def __pow__(self, k):
        return self.exponentiate(k)

    def concatenate(self, other: 'FiniteStateAutomaton'):
        """
        concatenate this FSA with another

        Parameters
        ----------
        other : FiniteStateAutomaton

        Returns
        -------
        FiniteStateAutomaton
        """
        msg = "you still need to implement FiniteStateAutomaton.concatenate"
        raise NotImplementedError(msg)

    def exponentiate(self, k: int) -> 'FiniteStateAutomaton':
        """
        concatenate this FSA k times

        Parameters
        ----------
        k : int
            the number of times to repeat; must be >1

        Returns
        -------
        FiniteStateAutomaton
        """
        if k <= 1:
            raise ValueError("must be >1")

        new = self

        for i in range(1,k):
            new += self

        return new


## Kleene Closure

Given an NFA $M = \langle Q, \Sigma, \delta, q_0, F \rangle$ recognizing $A = \mathbb{L}(M)$, the NFA $N = \text{kleene}(M) = \langle Q \cup \{q'_0\}, \Sigma, \delta', q'_0\not\in Q, F' \rangle$ recognizing $A^* = \mathbb{L}(N) = \mathbb{L}(\text{kleene}(M))$:

1. Define $$\delta'(q, \sigma) = \begin{cases}
q_0 & \text{if } (q = q'_0 \lor q \in F) \land \sigma = \epsilon \\
\delta(q, \sigma) & \text{otherwise} \\
\end{cases}$$



In [ ]:
class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """

    def kleene_star(self)  -> 'FiniteStateAutomaton':
        """
        construct the kleene closure machine

        Returns
        -------
        FiniteStateAutomaton
        """
        fsa = self._deepcopy()
        
        new_transition = fsa._transition_function
        new_transition.add_transitions({(s, ''): fsa._initial_state
                                        for s in fsa._final_states})

        return FiniteStateAutomaton(fsa._alphabet, fsa._states,
                                    fsa._initial_state, fsa._final_states,
                                    new_transition.transition_graph)

# Mapping between regular expressions and FSAs

It turns out that FSAs can generate exactly the same set of languages as regular expressions. To show this weak equivalence, we must show that every regular expression can be mapped to an equivalent FSA (specifically, NFA) and that every FSA can be mapped to an equivalent regular expression. 

This fact is useful for our purposes because (as we will see when we cover finite state transducers) it will provide the basis for compactly stating phonological rules using a regular expression-like metalanguage and *compiling* those rules to finite state automata. Thus, the mapping from regular expressions to FSAs will be the most important for us, though in principle the inverse mapping could be useful if we have an FSA and we wish to attempt to extract phonological generalizations from it. 

## Regular expressions to FSAs

Compilation takes advantages of the operations on FSAs that we just just covered.

$\text{compile}_\Sigma(\rho) = \begin{cases}\langle \{q_0\}, \Sigma, [], q_0, \{\} \rangle & \text{if } \rho = \emptyset\\\langle \{q_0\}, \Sigma, [], q_0, \{q_0\} \rangle & \text{if } \rho = \epsilon\\\langle \{q_0, q_1\}, \Sigma, [\langle q_0, \rho \rangle \rightarrow q_1], q_0, \{q_1\} \rangle & \text{if } \rho \in \Sigma\\\text{union}\left(\text{compile}_\Sigma(\rho_1), \text{compile}_\Sigma(\rho_2)\right) & \text{if } \rho = (\rho_1 \cup \rho_2)\\\text{concatenate}\left(\text{compile}_\Sigma(\rho_1), \text{compile}_\Sigma(\rho_2)\right) & \text{if } \rho = (\rho_1 \circ \rho_2)\\\text{kleene}\left(\text{compile}_\Sigma(\rho_0)\right) & \text{if } \rho = \rho_0^*\\\text{undefined} & \text{otherwise} \end{cases}$

## FSAs to regular expressions

A standard way of converting FSAs to regular expressions is to take an FSA generating a language on $\Sigma$ and mapping it to an FSA generating a regular expression using [*Kleene's algorithm*](https://en.wikipedia.org/wiki/Kleene%27s_algorithm). I will not cover the algorithm here. 



# Further operations

## Complementation

Given and NFA $M_1 = \langle Q_1, \Sigma, \delta_1, q_1, F_1 \rangle$ recognizing $A = \mathbb{L}(M_1)$, we construct NFA $M_2 = \langle Q_2, \Sigma, \delta_2, q_2, F_2 \rangle$ recognizes $\overline{A} = \mathbb{L}(M_2)$

1. Determinize $M_1$ to $M'_1 = \langle Q', \Sigma', \delta', q_0', F'_1 \rangle$ and define $Q_2 = Q'_1 \cup q_\text{sink}$ and $q_2 = q'_1$.
2. Define $\delta_2 = \mathrm{totalize}(\delta'_1)$, where $\mathrm{totalize}$ adds mappings to $q_\text{sink}$ where $\delta'_1$ is undefined
3. Define $F_2 = Q'_1 - F'_1$



In [ ]:
class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """

    def __neg__(self):
        return self.complement()

    def complement(self) -> 'FiniteStateAutomaton':
        """
        complement this FSA

        Returns
        -------
        FiniteStateAutomaton
        """
        fsa = self._deepcopy()
        fsa = fsa.determinize()
        fsa._transition_function.totalize(fsa._states, fsa._alphabet)
        fsa._final_states = fsa._states - fsa._final_states

        return fsa



## Intersection

Once we can compute the union and complementation of two machines, we get intersection for free. To see this, note that, according to [de Morgan's Laws](https://en.wikipedia.org/wiki/De_Morgan%27s_laws), $\overline{A \cup B} = \overline{A} \cap \overline{B}$. Thus, if we want the intersection $A \cap B = \overline{\overline{A \cap B}} = \overline{\overline{A} \cup \overline{B}}$.

The ability to compute intersection is very important for us because it allows us to express constraints on licit sequences. This ability in turn means that we can build grammars piecemeal. For instance, suppose we compute the Kleene closure for our syllable machine. We might view this as an English word machine. But the problem if that impossible words will end up generated by this machine: words with double consonants (geminates). 

The idea behind imposing a constraint in this context is that we can build another machine to filter geminates then intersect it with the word machine. How?



In [ ]:
class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """

    def __and__(self, other):
        return self.intersect(other)

    def intersect(self, other: 'FiniteStateAutomaton') -> 'FiniteStateAutomaton':
        """
        intersect this FSA with another

        Parameters
        ----------
        other : FiniteStateAutomaton

        Returns
        -------
        FiniteStateAutomaton
        """
        fsa1 = self.complement()
        fsa2 = other.complement()

        return fsa1.union(fsa2).complement()

# Recognition and Parsing

A recognizer for a grammar $G$ determines whether a string $\mathbf{s} \in \mathbb{L}(G)$. For FSAs, we can define a recognizer recursivesly.

$\text{recognize}_{\langle Q, \Sigma, \delta, q_0, F \rangle}(\mathbf{s}, q) = \begin{cases}\exists q''' \in \delta(\text{head}(\mathbf{s}), q) \cup \{q''\mid q' \in \delta_\epsilon(\epsilon, q) \text{ and } q'' \in \delta(\text{head}(\mathbf{s}), q')\}: \text{recognize}_{\langle Q, \Sigma, \delta, q_0, F \rangle}(\text{tail}(\mathbf{s}), q''') & \text{if } |\mathbf{s}| > 0\\\{q\} \cup \delta_\epsilon(\epsilon, q) \subseteq F & \text{if } |\mathbf{s}| = 0\\\end{cases}$

A parser looks very similar to a recognizer except that it additionally tracks the states that are traversed in recognizing the string. Because there are possibly multiple such traversals, a parser needs to produce a collection of such state traversal.

In [ ]:
FiniteStateAutomatonParse = List[Tuple[str, str]]


class FiniteStateAutomaton(FiniteStateAutomaton):
    """A finite state automaton

    Parameters
    ----------
    alphabet
    states
    initial_state
    final states
    transition_graph
    """
      def __call__(self, string: List[str], mode: str="recognize") -> Union[bool, List[FiniteStateAutomatonParse]]:
        """
        whether/how a string is accepted/parsed by the FSA

        Parameters
        ----------
        string
            the string to recognize or parse
        mode
            whether to run in "recognize" or "parse" mode
        """

        if mode == 'recognize':
            return self._recognize(string)
        elif mode == 'parse':
            return self._parse(string)
        else:
            msg = 'mode must be "recognize" or "parse"'
            raise ValueError(msg)

    @lru_cache(512)
    def _recognize(self, string: List[str], prev_state: Optional[str]=None) -> bool:
        """Whether a string is accepted by the FSA

        Parameters
        ----------
        string
            the string to recognize or parse
        """
        prev_states = {self._initial_state} if prev_state is None else {prev_state}
        prev_states |= {s for s in self._transition_function(prev_state, '')}
        
        if string:
            return any(self._recognize(string[1:], state)
                       for s in prev_states
                       for state in self._transition_function(s, string[0]))

        else:
            return any(s in self._final_states for s in prev_states)

    @lru_cache(512)
    def _parse(self, string: List[str], 
               prev_state: Optional[str]=None) -> List[List[Tuple[str, Optional[str]]]]:
        """How a string is parsed by the FSA
        
        This should return the list of transitions that 
        the machine could go through to parse a string

        Parameters
        ----------
        string
            the string to recognize or parse
        """
        prev_state = self._initial_state if prev_state is None else prev_state
        prev_states_epsilon = {s for s in self._transition_function(prev_state, '')}
        
        if string:
            parses = []
        
            for state in self._transition_function(prev_state, string[0]):
                parses.extend([[(prev_state, string[0])] + p 
                               for p in self._parse(string[1:], state)])
            
            for s in prev_states_epsilon:
                for state in self._transition_function(s, string[0]):
                    parses.extend([[(prev_state, ''), (s, string[0])] + p 
                                   for p in self._parse(string[1:], state)])
            
            return parses

        else:
            return [[] 
                    for s in {prev_state} | prev_states_epsilon 
                    if s in self._final_states]
 

# Transduction

So far, we have constructed morphophonological grammar in a [constraint-based](https://en.wikipedia.org/wiki/Model-theoretic_grammar) way. In this sort of approch, we define a set that contains all the strings we want (in the limit, $\Sigma^*$ itself); then, we filter out strings that violate a constraint (such as the one against gemination by complement and intersection operation. 

We can alterntively model morphophonological grammars using a [generative](https://en.wikipedia.org/wiki/Generative_grammar) approach. This could be accomplished using vanilla FSAs, like those discussed above; but it is often more productive to approach the problem as one of transduction: wherein we define some set of *underlying forms* that are mapped to surface forms by machines that implement morphophonological rules. These machines are generalizations of FSAs that we refer to as Finite State Transducers (FSTs). The main thing we add to an FSA to get an FSTs is that every edge is annotated with a mapping from a character in the input alphabet $\Sigma$ (or $\epsilon$) to one in the output alphabet $\Gamma$ (or $\epsilon$). The entire machine can then be viewed as a method for defining a relation between $\Sigma^*$ (the underlying forms) and $\Gamma^*$ (the surface forms). All the relations definable by an FST are known as the *regular relations*.

There are two ways that we can define FSTs: one that focuses on their use as methods for defining a relation and another that focuses on their use as methods for defining mappings from $\Sigma^*$ to $2^{\Gamma^*}$–i.e. as mappings from strings in $\Sigma^*$ to languages on $\Gamma$.

## The relation view

A Finite State Transducer (FST) is a grammar with 6 components:

1. A set of states $Q$
2. An input alphabet $\Sigma$
3. An output alphabet $\Gamma$
4. A transition function $\delta : Q \times (\Sigma \cup \{\epsilon\}) \times (\Gamma \cup \{\epsilon\}) \rightarrow \mathcal{P}(Q)$
5. An initial state $q_0$
6. A set of final states $F \subseteq Q$

## The mapping view

Alternatively, a Finite State Transducer (FST) is a grammar with 7 components:

1. A set of states $Q$
2. An input alphabet $\Sigma$
3. An output alphabet $\Gamma$
4. A transition function $\delta : Q \times (\Sigma \cup \{\epsilon\}) \rightarrow \mathcal{P}(Q)$
5. An output function $\omega : Q \times (\Sigma \cup \{\epsilon\}) \times Q \rightarrow \Gamma$
6. An initial state $q_0 \in Q$
7. A set of final states $F \subseteq Q$

FSTs in general are similar to vanilla FSAs:

1. Closed under union
2. Closed under concatenation
3. Closed under Kleene star

But they are different in a few respects:

1. Not all FSTs are determinizable.
2. FSTs aren't closed under intersection or complementation
3. Conceptualizing FSTs as language tuple recognizers/generators, we can define *projection*.
4. Conceptualizing FSTs as maps, we can define *composition* and *inversion*.